In Regression Tree the target variable is continuous (in Decision Tree the target variable is categorical)

In [1]:
from urllib.request import urlretrieve

url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%203/data/yellow_tripdata_2019-06.csv'
out_file = 'yellow_tripdata_2019-06.csv'

import os
if not os.path.exists(out_file):
    urlretrieve(url, out_file)
else:
    pass

In [2]:
import pandas as pd
df = pd.read_csv(out_file)
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-06-01 00:55:13,2019-06-01 00:56:17,1.0,0.0,1.0,N,145.0,145.0,2.0,3.0,0.5,0.5,0.00,0.0,0.3,4.30,0.0
1,1,2019-06-01 00:06:31,2019-06-01 00:06:52,1.0,0.0,1.0,N,262.0,263.0,2.0,2.5,3.0,0.5,0.00,0.0,0.3,6.30,2.5
2,1,2019-06-01 00:17:05,2019-06-01 00:36:38,1.0,4.4,1.0,N,74.0,7.0,2.0,17.5,0.5,0.5,0.00,0.0,0.3,18.80,0.0
3,1,2019-06-01 00:59:02,2019-06-01 00:59:12,0.0,0.8,1.0,N,145.0,145.0,2.0,2.5,1.0,0.5,0.00,0.0,0.3,4.30,0.0
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1.0,1.7,1.0,N,113.0,148.0,1.0,9.5,3.0,0.5,2.65,0.0,0.3,15.95,2.5


In [3]:
# preparing the dataset (it's for prediction of tip amount in taxis)

df = df[df['tip_amount'] != 0]  # remove rows with tip_amount = 0 (why?!)
df = df[df['tip_amount'] <= df['fare_amount']]  # we also remove some outliers, namely those where the tip was larger than the fare cost
df = df[(df['fare_amount'] >= 2) & (df['fare_amount'] < 200)]  # remove low and high fares

print(df['tpep_pickup_datetime'].dtype)
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])  # convert the text (object) to real datetime
print(df['tpep_pickup_datetime'].dtype)

df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])  # //

df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour  # extract the hour of pickup into new column
df['dropoff_hour'] = df['tpep_dropoff_datetime'].dt.hour

df['pickup_day'] = df['tpep_pickup_datetime'].dt.weekday  # extract day of week (0: Mon, 1: Tue...)
df['dropoff_day'] = df['tpep_dropoff_datetime'].dt.weekday

# extract trip duration as a new column
df['trip_time'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

clean_df = df.head(1000000)  # decreaset the data size, otherwise I'll get an error later
del df  # to free up memory and prevent accidental use of the previous df by deleting the DataFrame when it is no longer needed.

clean_df.head()

object
datetime64[ns]


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,pickup_hour,dropoff_hour,pickup_day,dropoff_day,trip_time
4,1,2019-06-01 00:03:25,2019-06-01 00:15:42,1.0,1.70,1.0,N,113.0,148.0,1.0,...,2.65,0.0,0.3,15.95,2.5,0,0,5,5,12.283333
5,1,2019-06-01 00:28:31,2019-06-01 00:39:23,2.0,1.60,1.0,N,79.0,125.0,1.0,...,1.00,0.0,0.3,14.30,2.5,0,0,5,5,10.866667
7,1,2019-06-01 00:54:49,2019-06-01 01:02:57,2.0,1.20,1.0,N,79.0,249.0,1.0,...,1.00,0.0,0.3,12.30,2.5,0,1,5,5,8.133333
9,1,2019-06-01 00:29:12,2019-06-01 01:03:13,1.0,8.60,1.0,N,186.0,243.0,1.0,...,7.05,0.0,0.3,42.35,2.5,0,1,5,5,34.016667
10,2,2019-06-01 00:01:48,2019-06-01 00:16:06,1.0,1.74,1.0,N,107.0,148.0,1.0,...,2.96,0.0,0.3,17.76,2.5,0,0,5,5,14.300000


In [5]:
# instead of ".concat" we can do this if we have both categoricals and continuous variables
new_df = clean_df.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'total_amount', 'tip_amount'], axis=1)  # remove the variables that we don't need
# define the categoricals
cat = ["VendorID","RatecodeID","store_and_fwd_flag","PULocationID", "DOLocationID","payment_type", "pickup_hour", "dropoff_hour", "pickup_day", "dropoff_day"]

X = pd.get_dummies(new_df, columns=cat)  # one-hot encode the cat while keeping the other columns
X = X.values  # no need to normalize or stnadardize (unlike what the main file says)

y = clean_df['tip_amount'].values.astype(float)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import time 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

t1 = time.time()  # to find out model training time
model_regTree = DecisionTreeRegressor(max_depth=8, random_state=32).fit(X_train, y_train)
t2 = time.time()
print(f'Model training time (s): {t2 - t1}')

model training time: 108.93383002281189


In [8]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

y_pred = model_regTree.predict(X_test)

MSE = mean_squared_error(y_test, y_pred)
print(MSE/np.var(y))  # the closer to 0, the better it is
print(f'RMSE: {np.sqrt(MSE)} | Rnage: {np.min(y)} - {np.max(y)}')
print(r2_score(y_test, y_pred))  # closer to 1 is better

0.22337880429044443
RMSE: 1.270456956980494 | Rnage: 0.01 - 91.0
0.7755958149846698
